# K₇ Spectral Gap v6: Rigorous Convergence Study

## Objectif

**Question scientifique** : Le produit λ₁ × H* converge-t-il vers une constante quand N → ∞ ?

**Méthodologie** :
1. Fixer un scaling k = c × N^α (pas de tuning pour atteindre un résultat)
2. Varier N et observer la convergence
3. Tester plusieurs valeurs de α pour trouver le régime de convergence
4. Reporter honnêtement le résultat, quel qu'il soit

## Théorie

Pour une variété de dimension d, la théorie spectrale des graphes suggère :
```
k_optimal ~ N^(2/(d+4))
```

Pour K₇ (d=7) : α_théorique = 2/11 ≈ 0.18

Nous testons : α ∈ {0.2, 0.3, 0.4, 0.5} pour voir lequel donne convergence.

In [ ]:
# Cell 1: Setup

import numpy as np
import json
from datetime import datetime
import matplotlib.pyplot as plt
import os
import time

try:
    import cupy as cp
    from cupyx.scipy.sparse import csr_matrix as cp_csr
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU_AVAILABLE = True
    print("GPU available via CuPy")
except ImportError:
    GPU_AVAILABLE = False
    from scipy.sparse import csr_matrix
    from scipy.sparse.linalg import eigsh
    cp = np
    print("CPU fallback")

# Constants (fixed, not tunable)
H_STAR = 99          # b₂ + b₃ + 1 = 21 + 77 + 1
DIM_G2 = 14          # dim(G₂)
DIM_K7 = 7           # dim(K₇)
TCS_RATIO = H_STAR / 84  # From topology, not fitted

# Theoretical predictions (for comparison, NOT targets)
PELL_PREDICTION = 14      # From 99² - 50×14² = 1
NUMERICAL_CLAIM = 13      # From previous numerical work

def clear_gpu():
    if GPU_AVAILABLE:
        cp.get_default_memory_pool().free_all_blocks()
        cp.get_default_pinned_memory_pool().free_all_blocks()

print(f"H* = {H_STAR}")
print(f"TCS ratio = {TCS_RATIO:.4f}")
print(f"Theoretical α = 2/(d+4) = 2/11 = {2/11:.4f}")
print(f"\nPell prediction: λ₁×H* = {PELL_PREDICTION}")
print(f"Previous numerical claim: λ₁×H* = {NUMERICAL_CLAIM}")
print(f"\nWe will NOT tune to match these - we observe what emerges.")

In [ ]:
# Cell 2: Core Functions (fixed implementation, no tunable parameters)

def sample_K7_TCS(N, seed):
    """Sample N points on K₇ via TCS construction.
    
    Fixed parameters from topology:
    - TCS ratio = H*/84 (not tunable)
    - S³ geodesic factor = 2 (geometric fact)
    """
    np.random.seed(seed)
    
    # Two S³ factors
    Q1 = np.random.randn(N, 4)
    Q1 /= np.linalg.norm(Q1, axis=1, keepdims=True)
    
    Q2 = np.random.randn(N, 4)
    Q2 /= np.linalg.norm(Q2, axis=1, keepdims=True)
    
    # S¹ factor (neck)
    theta = np.random.uniform(0, 2*np.pi, N)
    
    return Q1, Q2, theta

def compute_distances(Q1, Q2, theta):
    """Compute geodesic distance matrix on K₇."""
    # S³ geodesic: d = 2 × arccos(|q₁·q₂|)
    dot1 = np.abs(Q1 @ Q1.T)
    np.clip(dot1, 0, 1, out=dot1)
    d1 = 2.0 * np.arccos(dot1)
    
    dot2 = np.abs(Q2 @ Q2.T)
    np.clip(dot2, 0, 1, out=dot2)
    d2 = 2.0 * np.arccos(dot2)
    
    # S¹ geodesic (periodic)
    dtheta = np.abs(theta[:, None] - theta[None, :])
    dtheta = np.minimum(dtheta, 2*np.pi - dtheta)
    
    # TCS metric combination
    D = np.sqrt(d1**2 + (TCS_RATIO**2) * d2**2 + dtheta**2)
    
    return D

def compute_lambda1(D, k):
    """Compute first non-zero eigenvalue of normalized Laplacian.
    
    L = I - D^(-1/2) W D^(-1/2)
    
    Bandwidth σ = median of k-NN distances (automatic, not tuned).
    """
    N = D.shape[0]
    
    # Adaptive bandwidth from k-NN (principled, not tuned)
    knn_dists = np.partition(D, k+1, axis=1)[:, k]
    sigma = np.median(knn_dists)
    
    # Gaussian kernel
    W = np.exp(-D**2 / (2 * sigma**2))
    np.fill_diagonal(W, 0)
    
    # k-NN sparsification
    for i in range(N):
        idx = np.argpartition(D[i], k+1)[:k+1]
        mask = np.ones(N, dtype=bool)
        mask[idx] = False
        W[i, mask] = 0
    
    # Symmetrize
    W = (W + W.T) / 2
    
    # Normalized Laplacian
    deg = np.maximum(W.sum(axis=1), 1e-10)
    D_inv_sqrt = np.diag(1.0 / np.sqrt(deg))
    L = np.eye(N) - D_inv_sqrt @ W @ D_inv_sqrt
    
    # Compute eigenvalues
    if GPU_AVAILABLE:
        L_gpu = cp_csr(cp.array(L))
        eigs, _ = cp_eigsh(L_gpu, k=3, which='SA')
        eigs = cp.asnumpy(eigs)
    else:
        L_sparse = csr_matrix(L)
        eigs, _ = eigsh(L_sparse, k=3, which='SM')
    
    eigs = np.sort(eigs)
    return eigs[1], sigma  # First non-zero eigenvalue

print("Core functions defined (no tunable parameters).")

In [ ]:
# Cell 3: Convergence Study
# Test different α values and observe convergence behavior

print("="*70)
print("CONVERGENCE STUDY")
print("="*70)
print("\nScaling law: k = c × N^α")
print("We fix c=1 and test different α values.")
print("If λ₁×H* converges as N increases, the method is valid.")
print()

# Test configurations
N_values = [5000, 8000, 12000, 18000, 25000]
alpha_values = [0.20, 0.30, 0.40, 0.50]

# We use c such that k is reasonable (not too small, not too large)
# k should be in range [20, 200] for stability
# At N=25000, α=0.4 gives k = 25000^0.4 ≈ 50
# We use c=2 to get k ≈ 100 at N=25000 for α=0.4

results = {}

for alpha in alpha_values:
    print(f"\n{'='*70}")
    print(f"Testing α = {alpha}")
    print(f"{'='*70}")
    
    # Determine c such that k is reasonable
    # We want k ≈ 50-100 at our largest N
    c = 2.0  # Fixed multiplier
    
    alpha_results = []
    
    print(f"\n{'N':>7} | {'k':>5} | {'λ₁':>10} | {'σ':>8} | {'λ₁×H*':>10}")
    print("-" * 55)
    
    for N in N_values:
        # Compute k from scaling law
        k = max(15, int(c * (N ** alpha)))  # Minimum k=15 for stability
        
        t0 = time.time()
        
        # Sample and compute
        Q1, Q2, theta = sample_K7_TCS(N, seed=42)
        D = compute_distances(Q1, Q2, theta)
        lambda1, sigma = compute_lambda1(D, k)
        product = lambda1 * H_STAR
        
        elapsed = time.time() - t0
        
        print(f"{N:>7} | {k:>5} | {lambda1:>10.6f} | {sigma:>8.4f} | {product:>10.4f}")
        
        alpha_results.append({
            'N': N,
            'k': k,
            'lambda1': float(lambda1),
            'sigma': float(sigma),
            'product': float(product),
            'time_s': float(elapsed)
        })
        
        # Cleanup
        del Q1, Q2, theta, D
        clear_gpu()
    
    # Analyze convergence
    products = [r['product'] for r in alpha_results]
    
    # Check if converging (variance decreasing)
    first_half = products[:len(products)//2]
    second_half = products[len(products)//2:]
    var_first = np.var(first_half) if len(first_half) > 1 else 0
    var_second = np.var(second_half) if len(second_half) > 1 else 0
    
    converging = var_second < var_first
    final_value = np.mean(second_half)
    
    print(f"\nVariance (first half):  {var_first:.4f}")
    print(f"Variance (second half): {var_second:.4f}")
    print(f"Converging: {converging}")
    print(f"Final estimate: λ₁×H* ≈ {final_value:.2f}")
    
    results[alpha] = {
        'c': c,
        'data': alpha_results,
        'converging': bool(converging),
        'final_estimate': float(final_value),
        'var_first': float(var_first),
        'var_second': float(var_second)
    }

print("\n" + "="*70)
print("CONVERGENCE SUMMARY")
print("="*70)

In [ ]:
# Cell 4: Stability Test (multi-seed at best α)

print("="*70)
print("STABILITY TEST")
print("="*70)

# Find best α (lowest variance in second half)
best_alpha = min(results.keys(), key=lambda a: results[a]['var_second'])
print(f"\nBest α for convergence: {best_alpha}")
print(f"Final estimate at this α: {results[best_alpha]['final_estimate']:.4f}")

# Multi-seed test at fixed N with best α
N_test = 20000
k_test = max(15, int(2.0 * (N_test ** best_alpha)))
n_seeds = 5

print(f"\nStability test: N={N_test}, k={k_test}, {n_seeds} seeds")
print()

stability_results = []

for seed in range(n_seeds):
    Q1, Q2, theta = sample_K7_TCS(N_test, seed=seed)
    D = compute_distances(Q1, Q2, theta)
    lambda1, sigma = compute_lambda1(D, k_test)
    product = lambda1 * H_STAR
    
    print(f"  Seed {seed}: λ₁×H* = {product:.4f}")
    stability_results.append(product)
    
    del Q1, Q2, theta, D
    clear_gpu()

mean_stability = np.mean(stability_results)
std_stability = np.std(stability_results)

print(f"\nMean: {mean_stability:.4f} ± {std_stability:.4f}")
print(f"Coefficient of variation: {std_stability/mean_stability*100:.2f}%")

In [ ]:
# Cell 5: Analysis and Visualization

print("="*70)
print("FINAL ANALYSIS")
print("="*70)

# Compile final results
final_results = {
    'metadata': {
        'timestamp': datetime.now().isoformat(),
        'notebook': 'K7_Spectral_v6_Convergence.ipynb',
        'methodology': 'Rigorous convergence study without target fitting'
    },
    'constants': {
        'H_star': H_STAR,
        'TCS_ratio': float(TCS_RATIO),
        'dim_K7': DIM_K7,
        'dim_G2': DIM_G2
    },
    'theoretical_predictions': {
        'Pell_equation': PELL_PREDICTION,
        'previous_numerical': NUMERICAL_CLAIM,
        'theoretical_alpha': float(2/11)
    },
    'convergence_study': results,
    'stability_test': {
        'alpha': best_alpha,
        'N': N_test,
        'k': k_test,
        'results': [float(x) for x in stability_results],
        'mean': float(mean_stability),
        'std': float(std_stability)
    },
    'conclusions': {}
}

# Determine final answer
observed_value = mean_stability
dev_from_13 = abs(observed_value - 13) / 13 * 100
dev_from_14 = abs(observed_value - 14) / 14 * 100

final_results['conclusions'] = {
    'observed_lambda1_times_Hstar': float(observed_value),
    'uncertainty': float(std_stability),
    'deviation_from_13_pct': float(dev_from_13),
    'deviation_from_14_pct': float(dev_from_14),
    'best_alpha': float(best_alpha),
    'is_converging': bool(results[best_alpha]['converging']),
    'closest_to': 13 if dev_from_13 < dev_from_14 else 14
}

# Save results
os.makedirs('outputs', exist_ok=True)
with open('outputs/k7_spectral_v6_results.json', 'w') as f:
    json.dump(final_results, f, indent=2)
print("Saved: outputs/k7_spectral_v6_results.json")

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Convergence curves for different α
ax1 = axes[0, 0]
for alpha in alpha_values:
    Ns = [r['N'] for r in results[alpha]['data']]
    prods = [r['product'] for r in results[alpha]['data']]
    ax1.plot(Ns, prods, 'o-', label=f'α={alpha}', linewidth=2, markersize=6)

ax1.axhline(y=13, color='green', linestyle='--', linewidth=2, alpha=0.7, label='13 (numerical claim)')
ax1.axhline(y=14, color='orange', linestyle='--', linewidth=2, alpha=0.7, label='14 (Pell)')
ax1.set_xlabel('N (sample size)', fontsize=12)
ax1.set_ylabel('λ₁ × H*', fontsize=12)
ax1.set_title('Convergence Study: λ₁×H* vs N for different α', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: k vs N for different α
ax2 = axes[0, 1]
for alpha in alpha_values:
    Ns = [r['N'] for r in results[alpha]['data']]
    ks = [r['k'] for r in results[alpha]['data']]
    ax2.plot(Ns, ks, 'o-', label=f'α={alpha}')

ax2.set_xlabel('N', fontsize=12)
ax2.set_ylabel('k', fontsize=12)
ax2.set_title('Neighborhood size k vs N', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Stability test
ax3 = axes[1, 0]
ax3.bar(range(n_seeds), stability_results, color='steelblue', alpha=0.7)
ax3.axhline(y=mean_stability, color='red', linewidth=2, label=f'Mean: {mean_stability:.2f}')
ax3.axhline(y=13, color='green', linestyle='--', alpha=0.7, label='13')
ax3.axhline(y=14, color='orange', linestyle='--', alpha=0.7, label='14')
ax3.fill_between([-0.5, n_seeds-0.5], mean_stability-std_stability, mean_stability+std_stability, 
                  color='red', alpha=0.2)
ax3.set_xlabel('Seed', fontsize=12)
ax3.set_ylabel('λ₁ × H*', fontsize=12)
ax3.set_title(f'Stability Test (N={N_test}, k={k_test}, α={best_alpha})', fontsize=12)
ax3.legend()
ax3.set_xlim(-0.5, n_seeds-0.5)

# Plot 4: Final summary
ax4 = axes[1, 1]
categories = ['Observed', 'Claim (13)', 'Pell (14)']
values = [observed_value, 13, 14]
colors = ['steelblue', 'green', 'orange']
bars = ax4.bar(categories, values, color=colors, alpha=0.7)

# Add error bar on observed
ax4.errorbar(0, observed_value, yerr=std_stability, fmt='none', color='black', capsize=5, linewidth=2)

for bar, val in zip(bars, values):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 0.3,
             f'{val:.2f}', ha='center', va='bottom', fontsize=12, fontweight='bold')

ax4.set_ylabel('λ₁ × H*', fontsize=12)
ax4.set_title('Final Result Comparison', fontsize=12)
ax4.set_ylim(0, 16)

plt.tight_layout()
plt.savefig('outputs/k7_spectral_v6_results.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: outputs/k7_spectral_v6_results.png")

# Print conclusions
print("\n" + "="*70)
print("CONCLUSIONS")
print("="*70)
print(f"\n1. OBSERVED VALUE: λ₁ × H* = {observed_value:.4f} ± {std_stability:.4f}")
print(f"\n2. COMPARISON TO PREDICTIONS:")
print(f"   - Deviation from 13: {dev_from_13:.2f}%")
print(f"   - Deviation from 14: {dev_from_14:.2f}%")
print(f"   - Closest to: {final_results['conclusions']['closest_to']}")
print(f"\n3. CONVERGENCE:")
print(f"   - Best α for convergence: {best_alpha}")
print(f"   - Is converging: {results[best_alpha]['converging']}")
print(f"   - Theory predicts α = 2/11 ≈ 0.18")
print(f"\n4. INTERPRETATION:")
if dev_from_13 < 5 or dev_from_14 < 5:
    print(f"   The result is consistent with theoretical predictions.")
else:
    print(f"   The result differs from predictions - needs investigation.")